<a href="https://colab.research.google.com/github/PanoEvJ/Constitutional_BloomZ/blob/main/train_fast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install trl
# !pip install wandb
!pip install evaluate

In [2]:
!git clone https://github.com/PanoEvJ/Constitutional_BloomZ.git

Cloning into 'Constitutional_BloomZ'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 77 (delta 44), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (77/77), 47.76 KiB | 1.22 MiB/s, done.


In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!python Constitutional_BloomZ/train.py --HF_repo='PanoEvJ/gpt2-severe-detox-RLAIF'

Traceback (most recent call last):
  File "/content/Constitutional_BloomZ/train.py", line 19, in <module>
    from datasets import load_dataset
  File "/usr/local/lib/python3.10/dist-packages/datasets/__init__.py", line 43, in <module>
    from .arrow_dataset import Dataset
  File "/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py", line 65, in <module>
    from .arrow_reader import ArrowReader
  File "/usr/local/lib/python3.10/dist-packages/datasets/arrow_reader.py", line 28, in <module>
    import pyarrow.parquet as pq
  File "/usr/local/lib/python3.10/dist-packages/pyarrow/parquet/__init__.py", line 43, in <module>
    from pyarrow.fs import (LocalFileSystem, FileSystem,
  File "/usr/local/lib/python3.10/dist-packages/pyarrow/fs.py", line 60, in <module>
    initialize_s3()
KeyboardInterrupt
^C


In [ ]:
# HUGGING_FACE_USER_NAME = 'PanoEvJ'
# trained_model = "gpt2-detoxified-RLAIF----temp"

# model.push_to_hub(f"{HUGGING_FACE_USER_NAME}/{trained_model}", use_auth_token=True)

In [ ]:
!python Constitutional_BloomZ/eval.py --model_type="PanoEvJ/gpt2-detoxified-RLAIF"

In [4]:
from dataclasses import dataclass, field
from typing import Optional

import torch
from datasets import load_dataset
from torch.optim import Adam
from tqdm import tqdm
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    RobertaForSequenceClassification,
    RobertaTokenizer,
)

from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, create_reference_model, set_seed
from trl.core import LengthSampler

In [5]:
tqdm.pandas()

In [6]:
model_name      = "gpt2" # GPT2 or EleutherAI/gpt-j-6B
# log_with        = 'wandb'
learning_rate   =  (1.47e-5) * 2
mini_batch_size = 4
batch_size      = 16
gradient_accumulation_steps = 1
model_save_path = './'

In [7]:
config = PPOConfig(
    model_name=model_name,
    learning_rate=learning_rate,
    # log_with=log_with,
    ppo_epochs=100,
    mini_batch_size=mini_batch_size,
    batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
)

In [8]:
def build_dataset(
    config, dataset_name="allenai/real-toxicity-prompts", input_min_text_length=5, input_max_text_length=10
):
    """
    Build dataset for training. This builds the dataset from `load_dataset`, one should
    customize this function to train the model on its own dataset.

    Args:
        dataset_name (`str`):
            The name of the dataset to be loaded.

    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            The dataloader for the dataset.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token

    ds = load_dataset(dataset_name, split="train")

    def filter_fn(sample):
        toxicity = sample["prompt"]["severe_toxicity"]
        return toxicity is not None and toxicity > 0.6

    ds = ds.filter(filter_fn, batched=False)

    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        prompt = sample["prompt"]["text"]
        continuation = sample["continuation"]["text"]

        sample["input_ids"] = tokenizer.encode(prompt + continuation)[: input_size()]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")

    ds = ds.train_test_split(test_size=0.2, shuffle=False)["train"]

    return ds

In [9]:
# We retrieve the dataloader by calling the `build_dataset` function
min_input_length = 30
max_input_length = 40
dataset = build_dataset(config, input_min_text_length=min_input_length, input_max_text_length=max_input_length)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/allenai___json/allenai--real-toxicity-prompts-eb8779dd2693db47/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


Filter:   0%|          | 0/99442 [00:00<?, ? examples/s]

Map:   0%|          | 0/6744 [00:00<?, ? examples/s]

In [10]:
dataset

Dataset({
    features: ['filename', 'begin', 'end', 'challenging', 'prompt', 'continuation', 'input_ids', 'query'],
    num_rows: 5395
})

In [11]:
dataset['prompt'][0]

{'text': 'Corticosteroid drugs that are given by inhalers to children with',
 'identity_attack': tensor(0.6069),
 'flirtation': tensor(0.2329),
 'threat': tensor(0.6128),
 'insult': tensor(0.4965),
 'severe_toxicity': tensor(0.6285),
 'toxicity': tensor(0.7143),
 'profanity': tensor(0.4291),
 'sexually_explicit': tensor(0.1335)}

In [12]:
# dataset.push_to_hub("PanoEvJ/real-toxicity-prompts-severe0.7")

In [13]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

In [14]:
# set seed before initializing value head for deterministic eval
set_seed(config.seed)

In [15]:
# Now let's build the model, the reference model, and the tokenizer. We first load the model
# in bfloat16 to save memory using `transformers`.
model = AutoModelForCausalLM.from_pretrained(config.model_name, torch_dtype=torch.bfloat16)
# And then we pass the loaded model to `AutoModelForCausalLMWithValueHead`.
model = AutoModelForCausalLMWithValueHead.from_pretrained(model)

In [16]:
# We create a reference model by sharing 20 layers
ref_model = create_reference_model(model, num_shared_layers=6)

In [17]:
# We make sure to use `Adam` optimizer on the model parameters that require gradients.
optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=config.learning_rate)

In [18]:
# GPT-2 / GPT-J tokenizer has a pad token, but it is not eos_token by default. We need to set it to eos_token.
# only for this model.
tokenizer = AutoTokenizer.from_pretrained(config.model_name)
tokenizer.pad_token = tokenizer.eos_token

In [19]:
# We then build the PPOTrainer, passing the model, the reference model, the tokenizer
ppo_trainer = PPOTrainer(
    config,
    model,
    ref_model=ref_model,
    tokenizer=tokenizer,
    dataset=dataset,
    data_collator=collator,
    optimizer=optimizer,
)

In [20]:
# We then build the reward pipeline, we will use the toxicity model to compute the reward.
# We first load the toxicity model and tokenizer.
toxicity_model_id = "facebook/roberta-hate-speech-dynabench-r4-target"
toxicity_tokenizer = RobertaTokenizer.from_pretrained(toxicity_model_id)
# We load the toxicity model in fp16 to save memory.
toxicity_model = RobertaForSequenceClassification.from_pretrained(toxicity_model_id, torch_dtype=torch.float16).to(
    ppo_trainer.accelerator.device
)

In [21]:
# We then define the arguments to pass to the `generate` function. These arguments
# are passed to the `generate` function of the PPOTrainer, which is a wrapper around
# the `generate` function of the trained model.
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}

In [22]:
output_min_length = 20
output_max_length = 30
output_length_sampler = LengthSampler(output_min_length, output_max_length)

In [24]:
import numpy
import pandas as pd

sum_rewards = []
rewards_dict = {}
for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):

    query_tensors = batch["input_ids"]

    # Get response from the policy model
    response_tensors = []
    for query in query_tensors:
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze()[-gen_len:])
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    # Compute sentiment score # noqa
    texts = batch["response"]
    toxicity_inputs = toxicity_tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(
        ppo_trainer.accelerator.device
    )
    logits = toxicity_model(**toxicity_inputs).logits.float()
    toxicity_labels = (logits[:, 0]).tolist()

    rewards = [torch.tensor(output) for output in toxicity_labels]
    sum_rewards.append(numpy.mean(toxicity_labels))

    # Run PPO step
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

    # Save model every 100 epochs
    if epoch % 100 == 0:
        if ppo_trainer.accelerator.is_main_process:
            ppo_trainer.save_pretrained(model_save_path)
    
    if epoch == 1: 
        print(sum_rewards)
        print(list(range(1,epoch+1)))
        rewards_dict = {'epoch': list(range(1,epoch+2)), 'reward': sum_rewards}
        df = pd.DataFrame(rewards_dict) 
        df.to_csv('rewards.csv')
        break

1it [00:41, 41.12s/it]

[2.3349609375, 3.014129638671875]
[1]
